In [ ]:
!git clone https://github.com/dsa-playground/tvt2023.git
%cd /content/tvt2023/
!git pull
!pip install -r requirements.txt -t "tvt2023"

# Inleiding

### Use case 'Titanic'
Ruim honderd jaar geleden (1912) zonk de Titanic, vier uur nadat het schip op een ijsberg was gelopen. Slechts een derde deel van de opvarende overleefde deze ramp. Veel van deze gebeurtenis is vastgelegd, waaronder ook een dataset van passagiers. Deze dataset leent zich goed voor een introductie in de Data Science. Kan een algoritme voorspellen of een passagier overleeft? En, als je jezelf toevoegd, zou jij het dan overleefd hebben?

<!-- ![Laatste foto van de Titanic](../tvt2023/images/lastphoto_titanic.png) -->
<img src=../tvt2023/images/lastphoto_titanic.png alt="Laatste foto van de Titanic">

### Instructies omgeving
Voor deze workshop werken we in Google Colab. Dit is een online ontwikkelomgeving waarin je eenvoudig kunt experimenteren. Het notebook wat we voorbereid hebben staat al klaar. In een notebook staan cellen met ofwel code, tekst of afbeeldingen. Om een cel af te trappen (code draaien) zijn er twee mogelijkheden:
- Play-knop links van de cel
- Ctrl + Enter

### Imports & settings
Data Science heeft een sterke component met Computer Science. De meeste programmatuur zit op de achtergrond, maar om gebruik te maken van de functionaliteiten worden de functies en instellingen geladen in de cel hieronder. 

In [ ]:
# Imports
import pandas as pd
from scripts.preprocess.preprocess_frontend import zie_settings, laden_data, opschonen_data, numeriek_maken_data, voeg_passagiers_toe
from scripts.EDA.eda import basis_feiten, EDA_visualisaties, correlatie_heatmap
from scripts.modeling.modeling_frontend import train_and_save_model, voorspelling_genereren

# Settings
# settings for pandas
pd.set_option("display.max.columns",None) # alle kolommen tonen
pd.set_option("display.max.rows",500)    # eerste 500 rijen tonen
pd.set_option("display.precision", 2)     # precisie van de kolommen aanpassen
pd.set_option('display.float_format', lambda x: '{:.3f}'.format(x)) # floats output tot 3 decimalen
pd.set_option('display.max_colwidth', None)

In [ ]:
_config = zie_settings()

Data Science is geen doel op zich. Het doel is antwoord vinden voor een vraagstuk. Om te borgen dat Data (Science) producten aansluiten bij de wensen van een klant kan men het CRISP-DM proces (Cross-Industry Standard Process for Data Mining) volgen. 


<img src=../tvt2023/images/CRISP-DM.png width=400 height=400 alt="CRISP-DM">

Dit proces doorloopt de volgende stappen:
- Business understanding: Vinden van de hypothese en context.
- Data understanding: Verzamelen van relevante data.
- Data preparation: Aanpassen data zodat deze bruikbaar is voor algoritme.
- Modeling: Opzetten/inrichten algoritme om antwoord te vinden op hypothese. 
- Evaluation: Reflecteren of resultaat model hypothese verwerpt of aanneemt.
- Deployment: Naar productieomgeving brengen (zorg dragen dat model meermaals gebruikt kan worden). 

### Business Understanding
Het vraagstuk nu concentreert zich op wel/niet overleven van de Titanic. Oftewel:
- *Kan een algoritme voorspellen of een passagier de Titanic overleefd?*

En... door onszelf toe te voegen kijken of **wij** dit hadden overleefd!

### Data Understanding
Er is een dataset beschikbaar met informatie van passagiers. Er zijn twee datasets:
1. train: Dataset met passagiers én informatie of zij wel/niet overleefd hebben
2. test: Dataset met passagiers *zonder* informatie of zij wel/niet overleefd hebben
Laten we eens kijken naar de train dataset. Draai de code door op de playknop te drukken. 

In [ ]:
df_train, df_test = laden_data()
display(df_train.head())

Zoals je ziet is de dataset in het Engels en soms wat cryptisch weergegeven. Om het iets eenvoudiger te maken, transformeren we de dataset naar iets begrijpelijkere taal en zetten we vergelijkbare informatie bij elkaar.

In [ ]:
df_train_clean, df_test_clean = opschonen_data(df_train, df_test)
display(df_train_clean.head())

De data die we nu zien heeft informatie over:
* Passagier: ID, naam, geslacht, leeftijd, opstapplaats, aantal kinderen, aantal overige familieleden, totaal aantal familieleden
* Reisinformatie: Opstapplaats, ticket nummer, ticket klasse, cabine nummer
* Overleefd ja/nee

Wat we willen voorspellen is of mensen het overleefd hebben. De kolom 'Overleefd' is wat we noemen 'target-variabele'. De andere variabelen zijn mogelijk de verklarende variabelen. Om te kijken of er waarde zit in de variabelen, doen we een verkennende gegevensanalyse (EDA: exploratory data analysis).

In [ ]:
basis_feiten(df=df_train_clean)
EDA_visualisaties(df=df_train_clean)

### Data preparation
Gezien de dataset en deze visualisaties, kunnen we de dataset nog iets meer aanpassen zodat we dit kunnen toepassen met een model. Aanpassingen welke benodigd zijn:
- Verwijderen kolommen met geen relevante data (zoals 'ticket_nummer')
- Verwijderen kolommen met veel missende data (zoals 'cabine_nummer')
- Vullen van missende waarden waar mogelijk (zoals bij 'leeftijd')
- Afronden van leeftijd (34,5 jaar = 34 jaar)
- Numeriek maken van waarden (geslacht, opstapplaats, ticket_klasse, overleefd)
- Nieuwe index maken (unieke combinatie per rij)

Dit leidt tot de volgende datasets (train & test):

In [ ]:
df_train_num, df_test_num = numeriek_maken_data(df_train_clean, df_test_clean)
display(df_train_num.head(), df_test_num.head())

De dataset is bijna klaar voor het toepassen van een model, maar... we willen natuurlijk ook weten of je het zelf overleefd zou hebben! Laten we onszelf toevoegen aan de dataset.

In [ ]:
df_train_extended, df_test_extended = voeg_passagiers_toe(df_train_num, df_test_num)
display(df_train_extended.tail(), df_test_extended.tail())

### Modeling
Zoals in de presentatie besproken zijn er verschillende algoritmes om vraagstukken op te lossen. In deze workshop gaan we voor een classificatie-algoritme: KNN (K Nearest Neighbor). Hoe dit algoritme werkt laat zich het beste uitleggen op basis van een animatie:

<img src=../tvt2023/images/knn.gif alt="KNN">

bron: https://github.com/TomasBeuzen/machine-learning-tutorials/blob/master/ml-animations/gif/knn/knn.gif

Wat het algoritme doet is het bepaald tot welke groep een 'onbekend' punt behoort, door de dichtsbijzijnde punten te zoeken. De meerderheid van wat die punten zijn, bepalen wat het 'onbekende' punt wordt. Een variabele waar je in het trainen van een model mee experimenteert is bijvoorbeeld het aantal buren. 

Natuurlijk is dit een vereenvoudigd voorbeeld met slechts 2 variabelen (X1 en X2). In het geval van de Titanic dataset zijn er veel meer variabelen en dus dimensies. Wij als mensen kunnen tot 3 dimensies vrij aardig visualiseren. Een algoritme is natuurlijk niet geremd door meer dimensies. 

Laten we een model trainen met een aantal verschillende buren (van 1 tot 20). 

In [ ]:
# train_and_save_model(df=df_train_extended)

# Aanpassen functie zodat resultaten zichtbaar worden in DataFrame
# Willen we nog iets met classification report: ja
# Stukje uitleg over evaluatie --> daarna brug naar kiezen buren en voorspellen.

Nu kun je kiezen wat voor jou de meest logische keuze is voor het aantal buren.
Belangrijk: Weet dat het aantal buren:
* Geen waarde gelijk aan het aantal klassen dat het moet voorspellen.
* Geen veelvoud van het aantal klassen dat het moet voorspellen.

In het voorbeeld van de Titanic zijn er 2 klassen (overleefd/niet overleefd). Dus kies geen even getal.

Ps. Natuurlijk kun je dit automatiseren, voor de workshop hebben we dit niet gedaan voor het spelelement. 

In [ ]:
# Kies aantal buren en train een model daarop + voorspellen

In [ ]:
df_voorspelling = voorspelling_genereren(X=df_test_extended)
display(df_voorspelling.tail())

In [ ]:
# Zorg dragen voor leuk einde, bijv. leuke visualisatie (gehele populatie met 2 bestuurders highlight) 
# of 'je hebt de Titanic dan misschien niet overleefd, maar wel deze workshop!

In [ ]:
# To do:
# - Zorgen dat bepaalde warnings e.d. niet zichtbaar zijn. 
# - Zorgen dat we weten wat bijdraagt aan de kans van overleven. 

In [ ]:
# df_test_extended.tail()
# df_train_num.columns

In [ ]:
# testlist = df_train_num.columns
# m1 = ['Overleefd']
# not_in_testlist = list(set(testlist) - set(m1))
# not_in_testlist

In [ ]:
# from sklearn.model_selection import train_test_split
# X=df_train_num[['Geslacht', 'Leeftijd', 'Opstapplaats', 'Aantal_kinderen',
#        'Aantal_overige_familieleden', 'Ticket_klasse', 'Ticket_prijs']]
# y=df_train_num['Overleefd']
# # X_test = df_test_num[['Geslacht', 'Leeftijd', 'Opstapplaats', 'Aantal_kinderen',
# #        'Aantal_overige_familieleden', 'Ticket_klasse', 'Ticket_prijs']]
# X_train, X_test, y_train, y_test = train_test_split(X, y,
#     test_size = 0.2, random_state=1)

In [ ]:
# from sklearn.neighbors import KNeighborsClassifier
# from sklearn.metrics import accuracy_score

# acc = []

# for i in range(1,20):
#     knn = KNeighborsClassifier(n_neighbors = i)
#     knn.fit(X_train,y_train)
#     yhat = knn.predict(X_test)
#     acc.append(accuracy_score(y_test,yhat))
#     print("For k = ",i," : ",accuracy_score(y_test,yhat))

In [ ]:
# plt.figure(figsize=(8,6))
# plt.plot(range(1,20),acc, marker = "o")
# plt.xlabel("Value of k")
# plt.ylabel("Accuracy Score")
# plt.title("Finding the right k")
# plt.xticks(range(1,20))
# plt.show()
# max(acc)

In [ ]:
# acc.index(max(acc))

In [ ]:
# import plotly.express as px

# df = df = pd.DataFrame(dict(
#     Num_neighbours = range(1,20),
#     accuracy_score = acc
# ))
# fig = px.line(df, x="Num_neighbours", y="accuracy_score")
# fig.show()

In [ ]:
# KNN = KNeighborsClassifier(n_neighbors = 3)
# KNN.fit(X,y)
# y_pred = KNN.predict(X_test)
# df_KNN = pd.DataFrame()
# # df_KNN["PassengerId"] = test2["PassengerId"]
# # df_KNN["Survived"] = y_pred

In [ ]:
# X_test['Overleefd']=y_pred

In [ ]:
# X_test